### Example of Reshaping Data  from wide to long format

### Environment set up and data

In [1]:
# Import the relevant libraries...
# By the way, I am ussing Python 3
import pandas as pd
# Load the raw data using the ExcelFile object
data = pd.ExcelFile('reshaping_data.xlsx')

### Reusable Function

In [2]:
# Now define a function for parsing other funder targets data
def ReshapeFunc(excel_obj, i):
    """ Takes in an excel file object with multiple tabs in a wide format, and a specified index
    of the tab to be parsed and reshaped. Returns a dataframe of the specified tab
    reshaped to long format"""
    
    tabnames = data.sheet_names
    assert i < len(tabnames), "Your tab index exceeds the number of available tabs, try a lower number" 
    
    # parse and clean columns
    df = excel_obj.parse(sheetname=tabnames[i], skiprows=7)
    cols1 = [str(x)[:4] for x in list(df.columns)]
    cols2 = [str(x) for x in list(df.iloc[0,:])]
    cols = [x+"_"+y for x,y in zip(cols1,cols2)]
    df.columns = cols
    df = df.drop(["Unna_nan"], axis=1).iloc[1:,:].rename(columns={'dist_nan':'district',
                                                       'prov_nan': 'province',
                                                       'part_nan':'partner',
                                                       'fund_nan':'financing_source'})
    # new columns, drop some and change data type
    df['main_organization'] = tabnames[i].split("_")[0] + " "+ tabnames[i].split("_")[1]
    df.drop([c for c in df.columns if "Total" in c], axis=1, inplace= True) 
    
    for c in [c for c in df.columns if "yrs" in c]:
        df[c] = df[c].apply(lambda x: pd.to_numeric(x))

    # reshape - indexing, pivoting and stacking
    idx = ['district','province', 'partner','financing_source','main_organization']
    multi_indexed_df = df.set_index(idx)
    stacked_df = multi_indexed_df.stack(dropna=False)
    stacked_df.head(25) # check out the results!
    long_df = stacked_df.reset_index()
    
    # clean up and finalize
    col_str = long_df.level_5.str.split("_") 
    long_df['target_year'] = [x[0] for x in col_str] 
    long_df['target_age'] = [x[1] for x in col_str]
    long_df['target_quantity'] = long_df[0] # rename this column
    df_final = long_df.drop(['level_5', 0], axis=1)

    return df_final

### Use function, concatenate and save results

In [3]:
# Check that our function works:
check_df = ReshapeFunc(data, 2)
check_df.head(2)

,district,province,partner,financing_source,main_organization,target_year,target_age,target_quantity
0,District 19,Region 4,partner 3,Souce 4,OPQ inc,2017,10-14yrs,1
1,District 19,Region 4,partner 3,Souce 4,OPQ inc,2017,15-29yrs,974


In [4]:
# Apply the function iteratively, concatenate and save
dfs_list = [ReshapeFunc(data, i) for i in range(4)]
concat_dfs  = pd.concat(dfs_list)
concat_dfs.to_excel("reshaping_result_long_format.xlsx")